In [1]:
# プロジェクトの作成
import time, params, oci_client
from oci.ai_vision import AIServiceVisionClient
from oci.ai_vision.models import CreateProjectDetails, Project, AnalyzeImageDetails, AnalyzeImageResult, ImageClassificationFeature, InlineImageDetails, Label

vision_client = oci_client.get(AIServiceVisionClient) # type: AIServiceVisionClient

create_project_details = CreateProjectDetails(
    display_name = params.project_name,
    compartment_id = params.compartment_id
)

project = vision_client.create_project(create_project_details).data # type: Project

while True:
    latest_project = vision_client.get_project(project.id).data # type: Project
    if(latest_project.lifecycle_state == 'ACTIVE'):
        break
    print('.', end='')
    time.sleep(5)
print(F'\n"{project.display_name}" was created.')
time.sleep(5) # extra to go to the next cell

.....
"lemon_quality" was created.


In [2]:
# モデルの作成
import json
import params, oci_client
from oci.ai_vision.models import CreateModelDetails, Model, DataScienceLabelingDataset

project_id = project.id

with open('dataset_info.json') as f:
    dataset_info = json.load(f)

dataset = DataScienceLabelingDataset(
    dataset_id = dataset_info.get('id'),
    dataset_type = 'DATA_SCIENCE_LABELING'
)

create_model_details = CreateModelDetails(
    display_name = params.model_name,
    model_type = 'IMAGE_CLASSIFICATION',
    training_dataset = dataset,
    max_training_duration_in_hours = 24.0,
    project_id = project_id,
    compartment_id = params.compartment_id
)

vision_client = oci_client.get(AIServiceVisionClient) # type: AIServiceVisionClient
model = vision_client.create_model(create_model_details).data # type: Model
print(f'Started creating a new model "{model.display_name}"')

Started creating a new model "image_classification"


In [ ]:
# Model 情報を保存
import json

model_info ={
    "id" : model.id,
    "display_name" : model.display_name,
    "model_type" : model.model_type,
    "project_id" : model.project_id,
    "project_display_name" : project.display_name
}

with open(f'model_{params.model_name}.json', 'w') as f:
    json.dump(model_info, f, indent=2)

In [4]:
# モデルのステータスの確認
import json, params, oci_client
from oci.ai_vision import AIServiceVisionClient
from oci.ai_vision.models import Model

with open('model_info.json') as f:
    model_info = json.load(f)

vision_client = oci_client.get(AIServiceVisionClient) # type: AIServiceVisionClient
model = vision_client.get_model(model_info.get('id')).data # type: Model
print(f'Status of "{model.display_name}" is {model.lifecycle_state}.')
#print(model)


Status of "image_classification" is CREATING.
